#### Watch the Getting Started Quick Start [Video](https://aka.ms/SK-Getting-Started-Notebook)

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

**Step 1**: Import Semantic Kernel SDK from NuGet

In [19]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, *-*"

Installed Packages Microsoft.SemanticKernel, 0.14.547.1-preview

In [20]:
#r "nuget: Microsoft.SemanticKernel.Skills.Web, *-*"

Installed Packages Microsoft.SemanticKernel.Skills.Web, 0.14.547.1-preview

**Step 2**: Add your [Open AI Key](https://openai.com/api/) or [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) key to run a function below and save your key in the configuration files for later use.

In [21]:
// NOTE: When you run this section, you'll be prompted to enter your key

#!import config/Settings.cs

// Setup Open AI Configuration
bool useAzureOpenAI = true;
await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

// Uncomment this if you are using the Open AI Service and have an Org ID
// await Settings.AskOrg(useAzureOpenAI);

Settings: OK: Azure OpenAI endpoint configured [config/settings.json]
Settings: OK: deployment name configured [config/settings.json]
Settings: OK: API key configured [config/settings.json]


**Step 3**: Instantiate the Kernel

In [22]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Web;
using Microsoft.SemanticKernel.Skills.Web.Bing;

// Set Simple kernel instance
IKernel kernel = KernelBuilder.Create();

In [23]:
// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService(model, apiKey, orgId);

1. modelはAzure Open AI > Model deployments > Model deployment nameで取得できます。 

2. apiKeyとazureEndpointはAzure Open AI > Keys and Endpointで取得できます。 

In [ ]:
// Check the value
model

**Step 4**: Load and Run a Skill

In [25]:
#!import config/Env.cs

In [26]:
var bingConnector = new BingConnector(Env.bingApiKey);
kernel.ImportSkill(new WebSearchEngineSkill(bingConnector), "bing");

In [27]:
var question = "What's the largest building in the world?";
var bingResult = await kernel.Func("bing", "search").InvokeAsync(question);

In [28]:
//Check the value
bingResult.Result

Historically, the world's tallest man-made structure was the Great Pyramid of Giza in Egypt, which held the position for over 3800 years until the construction of Lincoln Cathedral in 1311. The Strasbourg Cathedral in France

- 「semantic-kernel/dotnet/src/Skills/Skills.Web/WebSearchEngineSkill.cs」は、BingSearch APIを呼び出して結果をJSON形式で返すWeb検索エンジンスキルです。

- BingSearchSkillは、次のプロンプトにbing.searchというIDで、マッピングされます。

```yaml
[COMMANDS AVAILABLE]
- bing.search
```

**Step 5**: Create and Run a Skill

In [29]:
using Microsoft.SemanticKernel.TemplateEngine;

In [30]:
const string SemanticFunction = @"Answer questions only when you know the facts or the information is provided.
When you don't have sufficient information you reply with a list of commands to find the information needed.
When answering multiple questions, use a bullet point list.
Note: make sure single and double quotes are escaped using a backslash char.

[COMMANDS AVAILABLE]
- bing.search

[INFORMATION PROVIDED]
{{ $externalInformation }}

[EXAMPLE 1]
Question: what's the biggest lake in Italy?
Answer: Lake Garda, also known as Lago di Garda.

[EXAMPLE 2]
Question: what's the biggest lake in Italy? What's the smallest positive number?
Answer:
* Lake Garda, also known as Lago di Garda.
* The smallest positive number is 1.

[EXAMPLE 3]
Question: what's Ferrari stock price ? Who is the current number one female tennis player in the world?
Answer:
{{ '{{' }} bing.search ""what\\'s Ferrari stock price?"" {{ '}}' }}.
{{ '{{' }} bing.search ""Who is the current number one female tennis player in the world?"" {{ '}}' }}.

[END OF EXAMPLES]

[TASK]
Question: {{ $input }}.
Answer: ";

- maxToken に指定する結果の長さは、モデルのトークン制限を超えないようにする必要があります。何も指定しなかった場合、デフォルトは 256 です。

[Azure Open AI Reference](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference) maxTokens: The maximum number of tokens to generate in the completion. The token count of your prompt plus max_tokens can't exceed the model's context length. Most models have a context length of 2048 tokens (except for the newest models, which support 4096). 

var oracle = kernel.CreateSemanticFunction(SemanticFunction, maxTokens: 200, temperature: 0, topP: 1);

In [31]:
var oracle = kernel.CreateSemanticFunction(SemanticFunction);

var context = kernel.CreateNewContext();
context["externalInformation"] = "";

var questions = "What's the exchange rate EUR:USD?";
Console.WriteLine(questions);

//context.Variables.Update(questions);
context["input"] = questions;
var answer = await oracle.InvokeAsync(context);

What's the exchange rate EUR:USD?


- BingSearchの検索結果に基づいて、AOAIが回答を返します。

In [32]:
answer

{{ bing.search "What's the exchange rate EUR:USD?" }}. Result {{ bing.search "What's the exchange rate EUR:USD?" }}. ErrorOccurred False LastErrorDescription LastException <null> CancellationToken System.Threading.CancellationToken IsCancellationRequested False CanBeCanceled False WaitHandle System.Threading.ManualResetEvent Handle 3176 SafeWaitHandle Microsoft.Win32.SafeHandles.SafeWaitHandle IsInvalid False IsClosed False Variables index value 0 [INPUT, {{ bing.search "What's the exchange rate EUR:USD?" }}.] Key INPUT Value {{ bing.search "What's the exchange rate EUR:USD?" }}. 1 [externalInformation, ] Key externalInformation Value Memory Microsoft.SemanticKernel.Memory.NullMemory Skills Microsoft.SemanticKernel.SkillDefinition.ReadOnlySkillCollection Log Microsoft.Extensions.Logging.Abstractions.NullLogger

In [33]:
answer.GetType()

Microsoft.SemanticKernel.Orchestration.SKContext

In [38]:

if (answer.ErrorOccurred) {
    Console.WriteLine("Error: Something was wrong...");
} else {
    if (answer.Result.Contains("bing.search", StringComparison.OrdinalIgnoreCase))
    {
        var promptRenderer = new PromptTemplateEngine();

        Console.WriteLine("---- Fetching information from Bing...");
        var information = await promptRenderer.RenderAsync(answer.Result, context);

        Console.WriteLine("Information found:");
        Console.WriteLine(information);
        Console.WriteLine(information.GetType());

        // The rendered prompt contains the information retrieved from search engines
        context["externalInformation"] = information;

        // Run the semantic function again, now including information from Bing
        context.Variables.Update(questions);
        answer = await oracle.InvokeAsync(context);
    }
    else
    {
        Console.WriteLine("AI had all the information, no need to query Bing.");
    }

    Console.WriteLine("---- ANSWER:");
    Console.WriteLine(answer);
}

---- Fetching information from Bing...
Information found:
Sorry, I don't have that information. Please use the 'bing.search' command to find the current exchange rate.
System.String
---- ANSWER:
Please use the 'bing.search' command to find the current exchange rate.


In [35]:
answer.GetType()

Microsoft.SemanticKernel.Orchestration.SKContext

In [36]:
answer

Sorry, I don't have that information. Please use the 'bing.search' command to find the current exchange rate. Result Sorry, I don't have that information. Please use the 'bing.search' command to find the current exchange rate. ErrorOccurred False LastErrorDescription LastException <null> CancellationToken System.Threading.CancellationToken IsCancellationRequested False CanBeCanceled False WaitHandle System.Threading.ManualResetEvent Handle 3176 SafeWaitHandle Microsoft.Win32.SafeHandles.SafeWaitHandle IsInvalid False IsClosed False Variables index value 0 [INPUT, Sorry, I don't have that information. Please use the 'bing.search' command to find the current exchange rate.] Key INPUT Value Sorry, I don't have that information. Please use the 'bing.search' command to find the current exchange rate. 1 [externalInformation, 1 Euro to US Dollar stats. Last 30 Days. Last 90 Days. High. These are the highest points the exchange rate has been at in the last 30 and 90-day periods. 1.1204. 1.1204. Low. These are the lowest points the exchange rate has been at in the last 30 and 90-day periods..] Key externalInformation Value 1 Euro to US Dollar stats. Last 30 Days. Last 90 Days. High. These are the highest points the exchange rate has been at in the last 30 and 90-day periods. 1.1204. 1.1204. Low. These are the lowest points the exchange rate has been at in the last 30 and 90-day periods.. Memory Microsoft.SemanticKernel.Memory.NullMemory Skills Microsoft.SemanticKernel.SkillDefinition.ReadOnlySkillCollection Log Microsoft.Extensions.Logging.Abstractions.NullLogger